# Website A/B Testing - Lab

## Introduction

In this lab, you'll get another chance to practice your skills at conducting a full A/B test analysis. It will also be a chance to practice your data exploration and processing skills! The scenario you'll be investigating is data collected from the homepage of a music app page for audacity.

## Objectives

You will be able to:
* Analyze the data from a website A/B test to draw relevant conclusions
* Explore and analyze web action data

## Exploratory Analysis

Start by loading in the dataset stored in the file 'homepage_actions.csv'. Then conduct an exploratory analysis to get familiar with the data.

> Hints:
    * Start investigating the id column:
        * How many viewers also clicked?
        * Are there any anomalies with the data; did anyone click who didn't view?
        * Is there any overlap between the control and experiment groups? 
            * If so, how do you plan to account for this in your experimental design?

In [12]:
# Your code for Exploratory Analysis 
import pandas as pd
import numpy as np

# Load the data
df = pd.read_csv('homepage_actions.csv')
df


,timestamp,id,group,action
0,2016-09-24 17:42:27.839496,804196,experiment,view
1,2016-09-24 19:19:03.542569,434745,experiment,view
2,2016-09-24 19:36:00.944135,507599,experiment,view
3,2016-09-24 19:59:02.646620,671993,control,view
4,2016-09-24 20:26:14.466886,536734,experiment,view
...,...,...,...,...
8183,2017-01-18 09:11:41.984113,192060,experiment,view
8184,2017-01-18 09:42:12.844575,755912,experiment,view
8185,2017-01-18 10:01:09.026482,458115,experiment,view
8186,2017-01-18 10:08:51.588469,505451,control,view


In [13]:
# Identify users who viewed and users who clicked
view_ids = df[df['action'] == 'view']['id']
click_ids = df[df['action'] == 'click']['id']

print(view_ids)
print(click_ids)

0       804196
1       434745
2       507599
3       671993
4       536734
         ...  
8183    192060
8184    755912
8185    458115
8186    505451
8187    461199
Name: id, Length: 6328, dtype: int64
8       349125
12      601714
15      487634
17      468601
21      555973
         ...  
8162    451198
8164    252195
8167    344770
8180    615849
8182    795585
Name: id, Length: 1860, dtype: int64


In [20]:
# Create a user-level DataFrame with a binary 'clicked' column.
user_data = df.groupby(['id', 'group'])['action'].apply(
    lambda x: 1 if 'click' in x.values else 0
).reset_index(name='clicked')
user_data

,id,group,clicked
0,182988,experiment,0
1,182994,control,1
2,183089,control,0
3,183136,experiment,0
4,183141,experiment,1
...,...,...,...
6323,937003,control,0
6324,937073,control,0
6325,937108,control,0
6326,937139,experiment,1


## Conduct a Statistical Test

Conduct a statistical test to determine whether the experimental homepage was more effective than that of the control group.

In [17]:
from scipy.stats import norm

# Filter user data into control and experiment groups
control_group = user_data[user_data['group'] == 'control']
experiment_group = user_data[user_data['group'] == 'experiment']

n_control = len(control_group)
n_experiment = len(experiment_group)

clicks_control = control_group['clicked'].sum()
clicks_experiment = experiment_group['clicked'].sum()


In [19]:
# Calculate pooled click-through rate
p_pooled = (clicks_control + clicks_experiment) / (n_control + n_experiment)
p_pooled

0.2939317319848293

In [26]:
# Calculate standard error and z-score
p_control = clicks_control / n_control
p_experiment = clicks_experiment / n_experiment
se_pooled = np.sqrt(p_pooled * (1 - p_pooled) * (1/n_control + 1/n_experiment))
z_score = (p_experiment - p_control) / se_pooled


In [27]:
# Calculate the p-value for a one-tailed test
p_value = 1 - norm.cdf(z_score)


In [28]:
print(f"Z-score: {z_score:.4f}")
print(f"P-value: {p_value:.4f}")

Z-score: 2.6186
P-value: 0.0044


## Verifying Results

One sensible formulation of the data to answer the hypothesis test above would be to create a binary variable representing each individual in the experiment and control group. This binary variable would represent whether or not that individual clicked on the homepage; 1 for they did and 0 if they did not. 

The variance for the number of successes in a sample of a binomial variable with n observations is given by:

## $n\bullet p (1-p)$

Given this, perform 3 steps to verify the results of your statistical test:
1. Calculate the expected number of clicks for the experiment group, if it had the same click-through rate as that of the control group. 
2. Calculate the number of standard deviations that the actual number of clicks was from this estimate. 
3. Finally, calculate a p-value using the normal distribution based on this z-score.

### Step 1:
Calculate the expected number of clicks for the experiment group, if it had the same click-through rate as that of the control group. 

In [29]:
#Your code here

# Control group click-through rate
p_control = clicks_control / n_control

# Expected clicks for experiment group
expected_clicks = n_experiment * p_control

print(f"Expected clicks for experiment group: {expected_clicks:.2f}")

Expected clicks for experiment group: 838.02


### Step 2:
Calculate the number of standard deviations that the actual number of clicks was from this estimate.

In [30]:
#Your code here
# Standard deviation of the binomial variable
std_dev = np.sqrt(n_experiment * p_control * (1 - p_control))

# Verified z-score
z_score_verified = (clicks_experiment - expected_clicks) / std_dev

print(f"Verified Z-score: {z_score_verified:.4f}")

Verified Z-score: 3.6625


### Step 3: 
Finally, calculate a p-value using the normal distribution based on this z-score.

In [31]:
#Your code here
from scipy.stats import norm

# Verified p-value
p_value_verified = 1 - norm.cdf(z_score_verified)

print(f"Verified P-value: {p_value_verified:.4f}")

Verified P-value: 0.0001


### Analysis:

Does this result roughly match that of the previous statistical test?

> Comment: **Yes, the results roughly match. Both the initial two-proportion z-test and the binomial verification method yielded extremely low p-values (0.0044 and 0.0001, respectively). Although the values are not identical, they are both well below the standard 0.05 significance level. This confirms that the observed increase in the click-through rate for the experimental homepage is statistically significant and not due to random chance.**

## Summary

In this lab, you continued to get more practice designing and conducting AB tests. This required additional work preprocessing and formulating the initial problem in a suitable manner. Additionally, you also saw how to verify results, strengthening your knowledge of binomial variables, and reviewing initial statistical concepts of the central limit theorem, standard deviation, z-scores, and their accompanying p-values.